In [1]:
import sys
sys.path.append('/home/hwang/anaconda3/lib/python3.7/site-packages/')

from sklearn import datasets
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import SimpleITK as sitk
import nibabel
import math
# import texture

from keras.utils.np_utils import to_categorical
from keras import optimizers 
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout, Input, Lambda, Reshape, AveragePooling2D, BatchNormalization
from keras.layers.merge import add, concatenate
from keras.applications.resnet50 import ResNet50, decode_predictions

from keras import backend as K
from keras.utils import plot_model

from keras.callbacks import EarlyStopping, LambdaCallback, ModelCheckpoint 
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization

from keras_unet.utils import plot_imgs, get_augmented, plot_segm_history
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance 
from keras_unet.models import custom_unet

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [2]:
import tensorflow as tf
print(tf.__version__)
tf.config.list_physical_devices('GPU')
from tensorflow.python.client import device_lib
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

2.4.1


In [3]:
import os
from keras_unet.metrics import iou, iou_thresholded

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.set_visible_devices(physical_devices[0:1], 'GPU')

In [4]:
# delete .DS_Store file in each folder 

# resample
            
resampath = '/home/gpuadmin/anaconda3/envs/py380/Work/Segmentation_UNet/T1NORM_RESAMPLE/HC/Train/'
resampfile = os.listdir(resampath)
resampfile.sort()
num_seg = len(resampfile)
count_seg = 0
for item_resamp in os.listdir(resampath):
    current_resamp = os.path.basename(item_resamp)
    if current_resamp == '.DS_Store':
        fullpath = os.path.join(resampath,current_resamp)
        os.remove(fullpath)

In [5]:
# feature for deep learning (3D): img

resampath = '/home/gpuadmin/anaconda3/envs/py380/Work/Segmentation_UNet/T1NORM_RESAMPLE/HC/Train/'
resampfile = os.listdir(resampath)
resampfile.sort()
num_img = len(resampfile)

first = sitk.ReadImage(os.path.join(resampath,resampfile[0]))
feature = sitk.GetArrayFromImage(first)

feature_reshape = feature.transpose(1,2,0)

for n in range(len(os.listdir(resampath))):

    i = sitk.ReadImage(os.path.join(resampath,resampfile[n]))
    j = sitk.GetArrayFromImage(i)
    j_reshape = j.transpose(1,2,0)
    feature_reshape = np.dstack((feature_reshape, j_reshape))

In [6]:
# delete .DS_Store file in each folder 

# resample
            
resampath = '/home/gpuadmin/anaconda3/envs/py380/Work/Segmentation_UNet/ROI_RAD_RESAMPLE/HC/Train/'
resampfile = os.listdir(resampath)
resampfile.sort()
num_seg = len(resampfile)
count_seg = 0
for item_resamp in os.listdir(resampath):
    current_resamp = os.path.basename(item_resamp)
    if current_resamp == '.DS_Store':
        fullpath = os.path.join(resampath,current_resamp)
        os.remove(fullpath)

In [7]:
# feature for deep learning (3D): mask

resampath = '/home/gpuadmin/anaconda3/envs/py380/Work/Segmentation_UNet/ROI_RAD_RESAMPLE/HC/Train/'
resampfile = os.listdir(resampath)
resampfile.sort()
num_img = len(resampfile)

first = sitk.ReadImage(os.path.join(resampath,resampfile[0]))
mask = sitk.GetArrayFromImage(first)

mask_reshape = mask.transpose(1,2,0)

for n in range(len(os.listdir(resampath))):

    i = sitk.ReadImage(os.path.join(resampath,resampfile[n]))
    j = sitk.GetArrayFromImage(i)
    j_reshape = j.transpose(1,2,0)
    mask_reshape = np.dstack((mask_reshape, j_reshape))

In [8]:
# Extract slices with masks identified from a training set 

mask_valid = np.zeros((feature_reshape.shape[0], feature_reshape.shape[1]))
feature_valid = np.zeros((feature_reshape.shape[0], feature_reshape.shape[1]))

for n in range(feature_reshape.shape[2]):

    mask_slice = mask_reshape[:,:,n]
    feature_slice = feature_reshape[:,:,n]
    
    if (len(np.unique(mask_slice)) > 1): # only if mask is identified by a radiologist 
        
        mask_valid = np.dstack((mask_valid, mask_slice))
        feature_valid = np.dstack((feature_valid, feature_slice))
        
mask_valid = mask_valid[:,:,1:]
feature_valid = feature_valid[:,:,1:]

print(mask_valid.shape)
print(feature_valid.shape)

(528, 528, 192)
(528, 528, 192)


In [9]:
feature = feature_valid
feature.shape
maxval = np.amax(feature)
feature_norm = feature/maxval
feature_temp = feature_norm.transpose(2,0,1)
feature_final = feature_temp.reshape((feature_temp.shape[0],feature_temp.shape[1],feature_temp.shape[2],1))
feature_final.shape

mask = mask_valid
mask_final = mask.transpose(2,0,1)
mask_final = mask_final.reshape((mask_final.shape[0],mask_final.shape[1],mask_final.shape[2],1))

In [10]:
X_train = feature_final
y_train = mask_final 

# feature_final = feature_reshape = []
# mask_final = mask_reshape = []

In [11]:
# delete .DS_Store file in each folder 

# resample
            
resampath = '/home/gpuadmin/anaconda3/envs/py380/Work/Segmentation_UNet/T1NORM_RESAMPLE/HC/Test/'
resampfile = os.listdir(resampath)
resampfile.sort()
num_seg = len(resampfile)
count_seg = 0
for item_resamp in os.listdir(resampath):
    current_resamp = os.path.basename(item_resamp)
    if current_resamp == '.DS_Store':
        fullpath = os.path.join(resampath,current_resamp)
        os.remove(fullpath)

In [12]:
# feature for deep learning (3D): img

resampath = '/home/gpuadmin/anaconda3/envs/py380/Work/Segmentation_UNet/T1NORM_RESAMPLE/HC/Test/'
resampfile = os.listdir(resampath)
resampfile.sort()
num_img = len(resampfile)

first = sitk.ReadImage(os.path.join(resampath,resampfile[0]))
feature = sitk.GetArrayFromImage(first)

feature_reshape = feature.transpose(1,2,0)

for n in range(len(os.listdir(resampath))):

    i = sitk.ReadImage(os.path.join(resampath,resampfile[n]))
    j = sitk.GetArrayFromImage(i)
    j_reshape = j.transpose(1,2,0)
    feature_reshape = np.dstack((feature_reshape, j_reshape))

In [13]:
# delete .DS_Store file in each folder 

# resample
            
resampath = '/home/gpuadmin/anaconda3/envs/py380/Work/Segmentation_UNet/ROI_RAD_RESAMPLE/HC/Test/'
resampfile = os.listdir(resampath)
resampfile.sort()
num_seg = len(resampfile)
count_seg = 0
for item_resamp in os.listdir(resampath):
    current_resamp = os.path.basename(item_resamp)
    if current_resamp == '.DS_Store':
        fullpath = os.path.join(resampath,current_resamp)
        os.remove(fullpath)

In [14]:
# feature for deep learning (3D): mask

resampath = '/home/gpuadmin/anaconda3/envs/py380/Work/Segmentation_UNet/ROI_RAD_RESAMPLE/HC/Test/'
resampfile = os.listdir(resampath)
resampfile.sort()
num_img = len(resampfile)

first = sitk.ReadImage(os.path.join(resampath,resampfile[0]))
mask = sitk.GetArrayFromImage(first)

mask_reshape = mask.transpose(1,2,0)

for n in range(len(os.listdir(resampath))):

    i = sitk.ReadImage(os.path.join(resampath,resampfile[n]))
    j = sitk.GetArrayFromImage(i)
    j_reshape = j.transpose(1,2,0)
    mask_reshape = np.dstack((mask_reshape, j_reshape))

In [15]:
# Extract slices with masks identified from a training set 

mask_valid = np.zeros((feature_reshape.shape[0], feature_reshape.shape[1]))
feature_valid = np.zeros((feature_reshape.shape[0], feature_reshape.shape[1]))

for n in range(feature_reshape.shape[2]):

    mask_slice = mask_reshape[:,:,n]
    feature_slice = feature_reshape[:,:,n]
    
    if (len(np.unique(mask_slice)) > 1): # only if mask is identified by a radiologist 
        
        mask_valid = np.dstack((mask_valid, mask_slice))
        feature_valid = np.dstack((feature_valid, feature_slice))
        
mask_valid = mask_valid[:,:,1:]
feature_valid = feature_valid[:,:,1:]

print(mask_valid.shape)
print(feature_valid.shape)

(528, 528, 187)
(528, 528, 187)


In [16]:
feature = feature_valid
feature.shape
maxval = np.amax(feature)
feature_norm = feature/maxval
feature_temp = feature_norm.transpose(2,0,1)
feature_final = feature_temp.reshape((feature_temp.shape[0],feature_temp.shape[1],feature_temp.shape[2],1))
feature_final.shape

mask = mask_valid
mask_final = mask.transpose(2,0,1)
mask_final = mask_final.reshape((mask_final.shape[0],mask_final.shape[1],mask_final.shape[2],1))

In [17]:
X_test = feature_final
y_test = mask_final 

# feature_final = feature_reshape = []
# mask_final = mask_reshape = []

In [18]:
train_gen = get_augmented(X_train, y_train, batch_size = 32, data_gen_args = dict(rotation_range = 5., width_shift_range = 0.05, height_shift_range = 0.05, shear_range = 40, zoom_range = 0.2, horizontal_flip = True, vertical_flip = True, fill_mode = 'constant'))
sample_batch = next(train_gen)
xx, yy = sample_batch
input_shape = X_train[0].shape

model = custom_unet(input_shape, filters = 32, use_batch_norm = True, dropout = 0.3, dropout_change_per_layer = 0.0, num_layers = 4)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 528, 528, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 528, 528, 32) 288         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 528, 528, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
spatial_dropout2d (SpatialDropo (None, 528, 528, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [19]:
batch_size = 32
model_filename = '2d_unet_BM.h5'
# es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1)
callback_checkpoint = ModelCheckpoint(model_filename, verbose = 1, monitor = 'val_loss', save_best_only=True)
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = [iou, iou_thresholded])

In [22]:
history = model.fit_generator(train_gen, steps_per_epoch = len(y_train)//batch_size, epochs = 10, validation_data = (X_test, y_test), callbacks = [callback_checkpoint])

Epoch 1/10


ResourceExhaustedError:  OOM when allocating tensor with shape[32,64,264,264] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/conv2d_3/Conv2D (defined at <ipython-input-20-42f5ca614be6>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_4570]

Function call stack:
train_function


In [ ]:
model.save_weights(model_filename)
model.load_weights(model_filename)
y_pred = model.predict(X_test)

In [ ]:
plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=19)